In [1]:
# -*- coding: utf-8 -*-
# Árbol de Decisión para predecir `exited` (churn) sobre Churn_Modelling-ETL 2.csv

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    RocCurveDisplay,
    precision_recall_curve,
    auc,
)

# ---------------------
# 1) Carga y preparación de datos
# ---------------------
BASE = Path("../../")              # Ajusta si lo necesitas
DATA = BASE / "Churn_Modelling-ETL 2.csv"

df = pd.read_csv(DATA)

target_col = "exited"
assert target_col in df.columns, "No se encontró la columna objetivo 'exited'."

# Eliminamos identificadores
drop_ids = [c for c in ["row_number", "customer_id", "surname"] if c in df.columns]
X = df.drop(columns=[target_col] + drop_ids, errors="ignore")
y = df[target_col].astype(int)

# Categóricas vs numéricas
cat_cols = [c for c in X.columns if X[c].dtype == "object"]
num_cols = [c for c in X.columns if c not in cat_cols]

# ---------------------
# 2) Preprocesamiento + modelo (Pipeline)
# ---------------------
# Nota: el árbol NO exige escalado, pero lo incluimos en el pipeline
# para dejar lista la misma estructura que usaremos con otros modelos.
preprocess = ColumnTransformer([
    ("num", StandardScaler(), num_cols),                                  # (opcional para árbol)
    ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), cat_cols),
])

# Hiperparámetros iniciales con ligera regularización para evitar sobreajuste
tree = DecisionTreeClassifier(
    criterion="gini",
    max_depth=6,                 # controla profundidad -> menos overfitting
    min_samples_split=50,        # requiere suficientes muestras para dividir
    min_samples_leaf=25,         # hojas mínimas para suavizar
    random_state=42
)

pipe = Pipeline([
    ("prep", preprocess),
    ("clf", tree),
])

# ---------------------
# 3) Train/Test split
# ---------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

# Entrenar
pipe.fit(X_train, y_train)

# ---------------------
# 4) Predicción y métricas (holdout)
# ---------------------
# Árbol no tiene predict_proba calibrada como ensambles, pero se puede usar proba de la hoja
y_score = pipe.predict_proba(X_test)[:, 1]
y_pred  = pipe.predict(X_test)

report = classification_report(y_test, y_pred, digits=4)
cm = confusion_matrix(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_score)

print("== MÉTRICAS (holdout) – Decision Tree ==")
print(report)
print(f"ROC-AUC: {roc_auc:.4f}")
print("Matriz de confusión:\n", cm)

# ---------------------
# 5) Gráficas de evaluación (Matplotlib)
# ---------------------
# ROC
plt.figure()
RocCurveDisplay.from_predictions(y_test, y_score)
plt.title(f"ROC – DecisionTree (AUC={roc_auc:.3f})")
plt.tight_layout()
plt.savefig("tree_roc.png", dpi=120)
plt.close()

# Precision–Recall
prec, rec, thr = precision_recall_curve(y_test, y_score)
pr_auc = auc(rec, prec)
plt.figure()
plt.plot(rec, prec)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title(f"Precision–Recall – DecisionTree (AUC={pr_auc:.3f})")
plt.tight_layout()
plt.savefig("tree_precision_recall.png", dpi=120)
plt.close()

# Matriz de confusión
plt.figure()
plt.imshow(cm, interpolation="nearest")
plt.title("Matriz de confusión – DecisionTree")
plt.colorbar()
ticks = np.arange(2)
plt.xticks(ticks, ["Pred 0","Pred 1"])
plt.yticks(ticks, ["Real 0","Real 1"])
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.tight_layout()
plt.savefig("tree_confusion_matrix.png", dpi=120)
plt.close()

# ---------------------
# 6) Importancia de variables (interpretabilidad)
# ---------------------
# Recuperamos nombres de features post-OHE para mapear importancias
ohe = pipe.named_steps["prep"].named_transformers_.get("cat")
num_names = num_cols
cat_names = []
if ohe is not None and hasattr(ohe, "get_feature_names_out"):
    cat_names = ohe.get_feature_names_out(cat_cols).tolist()
feature_names = num_names + cat_names

importances = pipe.named_steps["clf"].feature_importances_
n = min(len(importances), len(feature_names))
imp_df = pd.DataFrame({
    "feature": feature_names[:n],
    "importance": importances[:n]
}).sort_values("importance", ascending=False).head(20)

# Barra horizontal
plt.figure(figsize=(7,6))
plt.barh(imp_df["feature"], imp_df["importance"])
plt.gca().invert_yaxis()
plt.xlabel("Importancia")
plt.title("Top 20 features – DecisionTree")
plt.tight_layout()
plt.savefig("tree_feature_importance.png", dpi=120)
plt.close()

# ---------------------
# 7) (Opcional) Visualizar estructura del árbol
# ---------------------
# ¡Cuidado! Si hay muchas columnas tras OHE, el gráfico puede ser grande.
plt.figure(figsize=(16, 9))
plot_tree(
    pipe.named_steps["clf"],
    filled=True,
    max_depth=3,            # muestrario: primeros 3 niveles
    feature_names=feature_names,
    class_names=["stay(0)", "churn(1)"],
    proportion=True,
    rounded=True
)
plt.tight_layout()
plt.savefig("tree_structure_top3.png", dpi=120)
plt.close()

# ---------------------
# 8) (Opcional) Ajuste de umbral
# ---------------------
thr = 0.40  # ejemplo: mueve el umbral para ganar recall de churn
y_pred_thr = (y_score >= thr).astype(int)
print(f"\n== Métricas con umbral personalizado = {thr:.2f} ==")
print(classification_report(y_test, y_pred_thr, digits=4))
print("Matriz de confusión (umbral custom):\n", confusion_matrix(y_test, y_pred_thr))


== MÉTRICAS (holdout) – Decision Tree ==
              precision    recall  f1-score   support

           0     0.8759    0.9674    0.9193      1991
           1     0.7841    0.4637    0.5827       509

    accuracy                         0.8648      2500
   macro avg     0.8300    0.7155    0.7510      2500
weighted avg     0.8572    0.8648    0.8508      2500

ROC-AUC: 0.8496
Matriz de confusión:
 [[1926   65]
 [ 273  236]]

== Métricas con umbral personalizado = 0.40 ==
              precision    recall  f1-score   support

           0     0.8828    0.9568    0.9183      1991
           1     0.7485    0.5029    0.6016       509

    accuracy                         0.8644      2500
   macro avg     0.8156    0.7299    0.7600      2500
weighted avg     0.8554    0.8644    0.8538      2500

Matriz de confusión (umbral custom):
 [[1905   86]
 [ 253  256]]


<Figure size 640x480 with 0 Axes>